In [19]:
import pandas as pd
import numpy as np
import sklearn
from itertools import combinations

number_pickers = 4

orders = pd.read_csv('./OBSP_CARTNO.csv')
orders.head()


,ORD_NO,SKU_CD,NUM_PCS,LOC,CART_NO,SEQ
0,ORD_0199,SKU_0075,1,WP_0043,1,1
1,ORD_0199,SKU_0069,1,WP_0043,1,2
2,ORD_0001,SKU_0246,1,WP_0049,1,3
3,ORD_0199,SKU_0246,1,WP_0049,1,4
4,ORD_0227,SKU_0254,1,WP_0050,1,5


In [ ]:
carts_df = (orders.groupby('CART_NO').agg(WORKLOAD=('SKU_CD', 'count')).reset_index())
carts_df.head()

# carts_df.sort_values('WORKLOAD', ascending= False).iloc[0]


,CART_NO,WORKLOAD
0,1,16
1,2,14
2,3,17
3,4,12
4,5,13


In [ ]:
# orders[orders['CART_NO'] == 1]
# carts_df.shape[0] # 120

120

In [ ]:
assigned_carts = set()
batch_no = 1
n_pickers = number_pickers

while len(assigned_carts) < carts_df.shape[0] :
  unassigned = carts_df[~carts_df['CART_NO'].isin(assigned_carts)]
  highest_workload_cart_row = unassigned.sort_values('WORKLOAD', ascending= False).iloc[0] # CART_NO | WORKLOAD
  highest_workload_cartno = highest_workload_cart_row['CART_NO']
  highest_workload_quantity = highest_workload_cart_row['WORKLOAD']
  
  batch= []
  pickers = {}
  
  pickers[1] = [highest_workload_cartno]
  batch.append(highest_workload_cartno)
  assigned_carts.add(highest_workload_cartno)
  
  highest_workload_orders = orders[orders['CART_NO'] == highest_workload_cartno] # ORD_NO | SKU_CD | NUM_PCS | LOC | CART_NO 
  highest_workload_skus = highest_workload_orders['SKU_CD'].unique() # unique하는게 맞을듯! SKU_CD의 갯수 문제가 아니라 SKU_CD 종류로 인해 발생하는 작업량이 크게 늘어나는거라
  
  cart_similarities = []
  for _, row in unassigned.iterrows() :
    if row['CART_NO'] == highest_workload_cartno :
      continue
    candidate_orders = orders[orders['CART_NO'] == row['CART_NO']]
    candidate_skus = candidate_orders['SKU_CD'].unique()
    intersection = len(set(highest_workload_skus) & set(candidate_skus))
    union = len(set(highest_workload_skus) | set(candidate_skus))
    jaccard_sim = intersection / union if union > 0 else 0
    cart_similarities.append((row['CART_NO'], jaccard_sim, row['WORKLOAD'])) # CART_NO, jaccard_sim, WORKLOAD
    
  # 자카드 유사도를 기준으로 오름차순
  # 작업량 기준으로 내림차순
  # 유사도가 낮은 카트(피커별로 유사도가 낮게 배정해야까) + 유사도가 같다면 작업량이 높은 카트 순서
  cart_similarities.sort(key= lambda x : (x[1], -x[2])) # (59, 0,0, 18), (76, 0.0, 18), (82, 0.0, 18), ,,

  remaining_carts = [(c[0], c[2]) for c in cart_similarities] # 작업량 가장 높은 카트 제외한 CART_NO, WORKLOAD만 가져오기
  # print(remaining_carts) # [(59, 18), (76, 18), (82, 18), (112, 18), (3, 17), ,,]
  used_carts = set()
  
  for picker_id in range(2, n_pickers + 1) : # 2,3,4
    best_combo = None
    best_diff = float('inf') # 양의 최대치
    # 1~3개 조합 탐색
    for r in range(1, min(4, len(remaining_carts) + 1)) :  # (1, 최소 4) > 1,2,3 총 3개의 카트 배정해야 함
      for combo in combinations(remaining_carts, r) : # (CART_NO, WORKLOAD), 1,2,3 
        # print(combo) ((59, 18),) ,, ((59, 18), (76, 18), (82, 18))
        combo_carts = [c[0] for c in combo]
        combo_workload = sum(c[1] for c in combo)
        # print('combo_carts: ', combo_carts) # [59] [59, 76, 82]
        # print('combo_workload : ', combo_workload) # 18 54
        if any(c in used_carts for c in combo_carts) : 
          continue
        
        # 정해진 한 카트와 다른 카트와의 작업량 차이가 제일 작은 조합
        diff = abs(combo_workload - highest_workload_quantity)
        if diff < best_diff :
          best_diff = diff
          best_combo = combo_carts
        
        if diff == 0 :
          break
      if best_diff == 0 :
        break
    if best_combo : # best_combo_cartno
      pickers[picker_id] = best_combo # {2 : 작업량 제일 작은 조합}
      batch.extend(best_combo)
      used_carts.update(best_combo)
      assigned_carts.update(best_combo)
  
  for picker, carts in pickers.items() :
    for cart in carts : # 카트 조합
      orders.loc[orders['CART_NO'] == cart, 'PICKER_NO'] = picker
      orders.loc[orders['CART_NO'] == cart, 'BATCH_NO'] = batch_no # BATCH_NO : 카트들을 피커수에 맞게 묶은 배치 1(카트 N개가 피커 4명에게 배치되어 있음)
  batch_no += 1
      